In [1]:
# coding=utf-8
import pandas as pd
import csv  # Import csv
import tweepy
import re
import string
import nltk
import pandas as pd
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sinling import SinhalaTokenizer, POSTagger
# pd.set_option('display.max_colwidth',1000)
tokenizer = SinhalaTokenizer()

In [2]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-

import codecs
df = pd.read_csv('FinalListPreprocessedData.csv', header = None, encoding='utf-8', engine='python')
df.columns = ['ID','User','CREATED_AT','LABEL','stemming_words','LIKE COUNT','FOLLOWERS COUNT']
df = df.dropna()
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT
1,2.0,1.42E+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොය...,1,1433
2,3.0,1.42E+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා...,10,13898
3,4.0,1.42E+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87
4,5.0,1.42E+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87
5,6.0,1.42E+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87


In [3]:
tokenizer = SinhalaTokenizer()
def clean_text(text):
    tokenized_sentences = [tokenizer.tokenize(f'{ss}.') for ss in tokenizer.split_sentences(text)]
    tagger = POSTagger()
    pos_tags =tagger.predict(tokenized_sentences) 
    return pos_tags 



In [4]:
df['cleaned_msg'] = df['stemming_words'].apply(lambda x: clean_text(x))
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,cleaned_msg
1,2.0,1.42E+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොය...,1,1433,"[[(හාමුදුරුවරු, NNC), (සියලූම, NNC), (පූජකයන්ග..."
2,3.0,1.42E+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා...,10,13898,"[[(ද්විපාර්ශවික, NNP), (එකඟතා, NNP), (ජන, JJ),..."
3,4.0,1.42E+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ..."
4,5.0,1.42E+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ..."
5,6.0,1.42E+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ..."


In [5]:
def pos_tag(data):
    result = []
    if type(data) == tuple:
        return data[1]

    if type(data) == list:
        for inner in data:
            result.append(pos_tag(inner))
    return result 

In [6]:
df['pos_tagging'] = df['cleaned_msg'].apply(lambda x: pos_tag(x))
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,cleaned_msg,pos_tagging
1,2.0,1.42E+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොය...,1,1433,"[[(හාමුදුරුවරු, NNC), (සියලූම, NNC), (පූජකයන්ග...","[[NNC, NNC, NNC, JJ, NNC, VNF, VNN, NNJ, NNC, ..."
2,3.0,1.42E+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා...,10,13898,"[[(ද්විපාර්ශවික, NNP), (එකඟතා, NNP), (ජන, JJ),...","[[NNP, NNP, JJ, NNJ, NNC, RB, JCV, VNF, AUX, J..."
3,4.0,1.42E+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, POST, NNP, NNC, C..."
4,5.0,1.42E+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, RRPCV, VFM, FS]]"
5,6.0,1.42E+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, POST, FS]]"


In [7]:
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import re

# My work

In [8]:
def get_vec(text):
    x = [tokenizer.tokenize(f'{ss}.') for ss in tokenizer.split_sentences(text)]
    w2v_model = Word2Vec(min_count=1,window=2,vector_size=300,sample=1e-5,alpha=0.01,min_alpha=0.0007,negative=0,workers=2)
    w2v_model.build_vocab(x, progress_per=1)
    w2v_model.train(x, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)
    
    for index, key in enumerate(w2v_model.wv.key_to_index):
        vec = w2v_model.wv[key]
    return vec

In [9]:
df['vec'] = df['stemming_words'].apply(lambda x: get_vec(x))

In [10]:
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,cleaned_msg,pos_tagging,vec
1,2.0,1.42E+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොය...,1,1433,"[[(හාමුදුරුවරු, NNC), (සියලූම, NNC), (පූජකයන්ග...","[[NNC, NNC, NNC, JJ, NNC, VNF, VNN, NNJ, NNC, ...","[6.4769585e-05, 0.0007151119, 0.0003552143, 0...."
2,3.0,1.42E+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා...,10,13898,"[[(ද්විපාර්ශවික, NNP), (එකඟතා, NNP), (ජන, JJ),...","[[NNP, NNP, JJ, NNJ, NNC, RB, JCV, VNF, AUX, J...","[0.0016002202, -0.0012094593, -0.0014216034, 0..."
3,4.0,1.42E+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, POST, NNP, NNC, C...","[-0.00057380437, 0.0021353785, -0.0030921793, ..."
4,5.0,1.42E+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, RRPCV, VFM, FS]]","[-0.0016578555, -0.00042776822, 0.0010935458, ..."
5,6.0,1.42E+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, POST, FS]]","[0.0004441746, 0.0021802855, 0.003328201, 0.00..."


In [11]:
X = df['vec'].to_numpy()
X = X.reshape(-1,1)

In [12]:
X.shape

(532, 1)

In [13]:
X = np.concatenate(np.concatenate(X, axis = 0),axis = 0).reshape(-1,300)

In [14]:
y=df['LABEL']

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [16]:
X_train.shape, X_test.shape

((425, 300), (107, 300))

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train,y_train)

LinearSVC()

In [18]:
y_pred = clf.predict(X_test)

In [19]:
from sklearn.metrics import classification_report
from sklearn import metrics

print(metrics.confusion_matrix(y_test,y_pred))

print(metrics.classification_report(y_test,y_pred))



[[ 0  0 18  0  0]
 [ 0  0  4  0  0]
 [ 0  0 42  0  0]
 [ 0  0 22  0  0]
 [ 0  0 21  0  0]]
               precision    recall  f1-score   support

        Crime       0.00      0.00      0.00        18
Entertainment       0.00      0.00      0.00         4
        Other       0.39      1.00      0.56        42
     Politics       0.00      0.00      0.00        22
        Sport       0.00      0.00      0.00        21

     accuracy                           0.39       107
    macro avg       0.08      0.20      0.11       107
 weighted avg       0.15      0.39      0.22       107



c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont